In [1]:
"""
Teesside University 
MSc Applied Data Science with advanced practice
Machine Learning ICA
---
Name: AGUAGWA EMMANUEL UCHENNA
Student Number: A0436807
Email: A0436807@live.tees.ac.uk

Machine Learning techniques to Predict stroke.
From https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?datasetId=1120859&sortBy=voteCount

Stroke, alternatively called brain assault, occurs when the blood supply to the brain is cut off, 
depriving it of oxygen and nutrients and resulting in the death of neurons within hours.

Notebook organisation:
1. Import libraries and data.
2. Distribution of Targets.
3. Data cleaning.
4. Exploratory Data Analysis.
5. Data visuals.
6. Machine learning.
8. Summary.
"""

'\nTeesside University \nMSc Applied Data Science with advanced practice\nMachine Learning ICA\n---\nName: AGUAGWA EMMANUEL UCHENNA\nStudent Number: A0436807\nEmail: A0436807@live.tees.ac.uk\n\nMachine Learning techniques to Predict stroke.\nFrom https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?datasetId=1120859&sortBy=voteCount\n\nStroke, alternatively called brain assault, occurs when the blood supply to the brain is cut off, \ndepriving it of oxygen and nutrients and resulting in the death of neurons within hours.\n\nNotebook organisation:\n1. Import libraries and data.\n2. Distribution of Targets.\n3. Data cleaning.\n4. Exploratory Data Analysis.\n5. Data visuals.\n6. Machine learning.\n8. Summary.\n'

In [2]:
###1.1 Libraries and Utilities

In [3]:
import warnings 
warnings.filterwarnings('ignore')

# basic libraries
import os
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
import time

#visulaization modules
import missingno as msno
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
!pip install pywaffle
from pywaffle import Waffle

%matplotlib inline
init_notebook_mode(connected= True)




#Common model helpers
from sklearn.preprocessing import (StandardScaler,
                                   LabelEncoder,
                                   OneHotEncoder)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, 
                             auc, 
                             precision_score,
                             recall_score,
                             f1_score, 
                             roc_auc_score,
                             confusion_matrix,
                            classification_report,
                            roc_curve)
from sklearn.model_selection import (GridSearchCV,
                                     StratifiedKFold,
                                     cross_val_score)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split, cross_val_score



# dimensionality reduction
from sklearn.decomposition import PCA
from umap.umap_ import UMAP
import pylab as pl

# imbalance dataset handling

from imblearn.datasets import make_imbalance
from imblearn.under_sampling import (RandomUnderSampler, 
                                     ClusterCentroids,
                                     TomekLinks,
                                     NeighbourhoodCleaningRule,
                                     EditedNearestNeighbours,
                                     NearMiss)


from imblearn.over_sampling import (SMOTE,
                                    ADASYN)




# model algorithams
from sklearn.ensemble import (RandomForestClassifier, 
                              AdaBoostClassifier, 
                              GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


pd.set_option('display.max_columns', None)



In [4]:
#1.2 Data Preprocessing

In [5]:
#Loading data


stroke_df = pd.read_csv('/Users/apple/Documents/TU/second semester/Machine learning/STROKE/ICA work/healthcare-dataset-stroke-data.csv', delimiter = ',', encoding = 'utf-8')
stroke_df.head(5).T

FileNotFoundError: [Errno 2] No such file or directory: '/Users/apple/Documents/TU/second semester/Machine learning/STROKE/ICA work/healthcare-dataset-stroke-data.csv'

In [ ]:
#Non-Null ValueCounts and Feature Datatypes
stroke_df.info()

In [ ]:
##Statistics of Categorical and Numerical Data

In [ ]:
# statistics of numerical data
round (stroke_df.describe(exclude = 'object'), 3)

In [ ]:
# statistics of categorical data
round (stroke_df.describe(exclude = ['float', 'int64']),2)

In [ ]:
##Visualization of Nullity of the Dataset

In [ ]:
color = ['#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#d0e0f2','#08306b']

fig, ax = plt.subplots(figsize = (12,4), dpi = 70)
fig.patch.set_facecolor('#f6f5f5')
ax.set_facecolor('#f6f5f5')


msno.bar(stroke_df, sort = 'descending', 
         color = color, 
         ax = ax, fontsize =8,
         labels = 'off',filter = 'top')

ax.text(-1,1.35,'Visualization of Null value',{'font': 'times new roman', 'Size': 24,  'color':'black'},alpha = 0.9)
ax.text(-1,1.2,'There are a total of 5110 datapoints in the supplied dataset. \nThe only feature with null values is "bmi.".',{'font': 'times new roman', 'Size': 12,  'color':'black'}, alpha = 0.7)

ax.set_xticklabels(ax.get_xticklabels(),rotation = 90, 
                   ha = 'center', **{'font': 'times new roman', 'Size': 14,'weight':'normal','color':'#1434A4'}, alpha = 1)
ax.set_yticklabels('')
ax.spines['bottom'].set_visible(True)

fig.show()

In [ ]:
#MISSING Value
stroke_df.isnull().sum()

In [ ]:
# handling missing values
stroke_df['bmi'] = stroke_df['bmi'].fillna(round (stroke_df['bmi'].median(), 2))
stroke_df.isnull().sum()

In [ ]:
## binning of numerical variables

stroke_df['bmi_cat'] = pd.cut(stroke_df['bmi'], bins = [0, 19, 25,30,10000], labels = ['Underweight', 'Ideal', 'Overweight', 'Obesity'])
stroke_df['age_cat'] = pd.cut(stroke_df['age'], bins = [0,13,18, 45,60,200], labels = ['Children', 'Teens', 'Adults','Mid Adults','Elderly'])
stroke_df['glucose_cat'] = pd.cut(stroke_df['avg_glucose_level'], bins = [0,90,160,230,500], labels = ['Low', 'Normal', 'High', 'Very High'])

In [ ]:
#2.0 Distribution of Targets

In [ ]:
eda = pd.DataFrame( stroke_df.groupby(['stroke'])['stroke'].count())

# plot
fig, ax = plt.subplots(figsize = (6,6), dpi = 70)
ax.barh([1], eda.stroke[1], height = 0.7, color = '#d0e0f2')
plt.text(-1150,-0.08, 'Healthy',{'font': 'times new roman','weight':'bold','Size': '16','style':'normal', 'color':'#08306b'})
plt.text(5000,-0.08, '95%',{'font':'times new roman','weight':'bold' ,'size':'16','color':'#08306b'})
ax.barh([0], eda.stroke[0], height = 0.7, color = '#08306b')
plt.text(-1000,1, 'Stroke', {'font': 'times new roman','weight':'bold','Size': '16','style':'normal', 'color':'#d0e0f2'})
plt.text(300,1, '5%',{'font':'times new roman', 'weight':'bold','size':'16','color':'#d0e0f2'})

fig.patch.set_facecolor('#f6f5f5')
ax.set_facecolor('#f6f5f5')

plt.text(-1150,1.77, 'Percentage of Individuals with Strokes' ,{'font': 'times new roman', 'Size': '25','weight':'bold', 'color':'black'})
plt.text(4650,1.65, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
plt.text(5650,1.65, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
plt.text(5750,1.65, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})
plt.text(-1150,1.5, 'It is a very imbalanced distribution, \nand it is evident that 5% of the population is prone \nto heart attacks.', 
        {'font':'times new roman', 'size':'12.5','color': 'black'})

ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(True)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)


In [ ]:
#2.1 Univariate analysis of continuous variables


fig = plt.figure(figsize = (24,10), dpi = 60)

gs = fig.add_gridspec(10,24)
gs.update(wspace = 1, hspace = 0.05)


ax2 = fig.add_subplot(gs[1:4,0:8]) #distribution plot
ax3 = fig.add_subplot(gs[6:9, 0:8]) #hue distribution plot
ax1 = fig.add_subplot(gs[1:10,13:]) #dumbbell plot

# axes list
axes = [ ax1,ax2, ax3]

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_yaxis().set_visible(False)
    ax.set_facecolor('#f6f5f5')
    
    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

fig.patch.set_facecolor('#f6f5f5')
        
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(True)

# dumbbell plot of stroke and healthy people

stroke_age = stroke_df[stroke_df['stroke'] == 1].age_cat.value_counts()
healthy_age = stroke_df[stroke_df['stroke'] == 0].age_cat.value_counts()

ax1.hlines(y = ['Children', 'Teens', 'Adults', 'Mid Adults', 'Elderly'], xmin = [644,270,1691,1129,1127], 
          xmax = [1,1,11,59,177], color = 'grey',**{'linewidth':0.5})


sns.scatterplot(y = stroke_age.index, x = stroke_age.values, s = stroke_age.values*2, color = '#d0e0f2', ax= ax1, alpha = 1)
sns.scatterplot(y = healthy_age.index, x = healthy_age.values, s = healthy_age.values*2, color = '#08306b', ax= ax1, alpha = 1)

ax1.axes.get_xaxis().set_visible(False)
ax1.set_xlim(xmin = -500, xmax = 2250)
ax1.set_ylim(ymin = -1,ymax = 5)

ax1.set_yticklabels( labels = ['Children', 'Teens', 'Adults', 'Mid Adults', 'Elderly'],fontdict = {'font':'times new roman', 'fontsize':16,'fontweight':'bold', 'color':'black'})

ax1.text(-950,5.8, 'How Age Impact on Having Strokes?' ,{'font': 'times new roman', 'Size': '25','weight':'bold', 'color':'black'},alpha = 0.9)
ax1.text(1000,4.8, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
ax1.text(1300,4.8, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
ax1.text(1350,4.8, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})
ax1.text(-950,5., 'Age has a considerable influence on the incidence of strokes, \nand it is evident that the incidence of strokes is highest among the elderly and middle-aged, \nwhile it is minimal among younger individuals.', 
        {'font':'times new roman', 'size':'16','color': 'black'})

ax1.text(stroke_age.values[0] + 30,4.05, stroke_age.values[0], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_age.values[2] - 300,4.05, healthy_age.values[2], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})

ax1.text(stroke_age.values[1] + 30,3.05, stroke_age.values[1], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_age.values[1] - 300,3.05, healthy_age.values[1], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})




# distribution plots ---- only single variable

sns.kdeplot(data = stroke_df, x = 'age', ax = ax2, shade = True, color = '#1434A4', alpha = 1, )
ax2.set_xlabel('Age of a person', fontdict = {'font':'times new roman', 'color': 'black', 'size': 16,'weight':'bold' })
ax2.text(-17,0.025,'Overall Age Distribution - How skewed is it?', {'font':'times new roman', 'color': 'black','weight':'bold','size':24}, alpha = 0.9)
ax2.text(-17,0.021, 'Based on Age, we have data ranging from babies to the elderly. \nThe adult population constitutes the middle group.', 
        {'font':'times new roman', 'size':'16','color': 'black'})

ax2.text(80,0.019, 'Total',{'font':'times new roman', 'size':'14','color': '#2c003e','weight':'bold'})
ax2.text(92,0.019, '=',{'font':'times new roman', 'size':'14','color': 'black','weight':'bold'})
ax2.text(97,0.019, 'Stroke',{'font':'times new roman', 'size':'14','color': '#d0e0f2','weight':'bold'})
ax2.text(113,0.019, '+',{'font':'times new roman', 'size':'14','color': 'black','weight':'bold'})
ax2.text(117,0.019, 'Healthy',{'font':'times new roman', 'size':'14','color': '#08306b','weight':'bold'})



# distribution plots with hue of strokes


sns.kdeplot(data = stroke_df[stroke_df['stroke'] == 0], x = 'age',ax = ax3, shade = True,  alpha = 1, color = '#08306b' )
sns.kdeplot(data = stroke_df[stroke_df['stroke'] == 1], x = 'age',ax = ax3, shade = True,  alpha = 0.8, color = '#d0e0f2')

ax3.set_xlabel('Age of a person', fontdict = {'font':'times new roman', 'color': 'black', 'weight':'bold','size': 16})

ax3.text(-17,0.0525,'Age-Stroke Distribution - How serious is it?', {'font':'times new roman', 'weight':'bold','color': 'black', 'size':24}, alpha= 0.9)
ax3.text(-17,0.043,'It is evident, based on the distribution of strokes, that older persons \nexperience a considerable number of strokes.', {'font':'times new roman', 'color': 'black', 'size':14})
ax3.text(100,0.043, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
ax3.text(117,0.043, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
ax3.text(120,0.043, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})

fig.text(0.25,1,'Heart Attacks and Age: The Story of a Heart that Has Aged',{'font':'times new roman', 'weight':'bold','color': 'black', 'size':35})
fig.show()


In [ ]:
# sugar distribution plots


fig = plt.figure(figsize = (24,10), dpi = 60)

gs = fig.add_gridspec(10,24)
gs.update(wspace = 1, hspace = 0.05)


ax2 = fig.add_subplot(gs[0:3,0:10]) #distribution plot
ax3 = fig.add_subplot(gs[5:10, 0:10]) #hue distribution plot
ax1 = fig.add_subplot(gs[0:,13:]) #dumbbell plot

# axes list
axes = [ ax1,ax2, ax3]

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_yaxis().set_visible(False)
    ax.set_facecolor('#f6f5f5')
    
    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

fig.patch.set_facecolor('#f6f5f5')
        
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(True)

# dumbbell plot of stoke and healthy people

stroke_glu = stroke_df[stroke_df['stroke'] == 1].glucose_cat.value_counts()
healthy_glu = stroke_df[stroke_df['stroke'] == 0].glucose_cat.value_counts()

ax1.hlines(y = ['Low', 'Normal', 'High', 'Very High'], xmin = [2316,1966,478,101], 
          xmax = [89,71,71,18], color = 'grey',**{'linewidth':0.5})


sns.scatterplot(y = stroke_glu.index, x = stroke_glu.values, s = stroke_glu.values, color = '#d0e0f2', ax= ax1, alpha = 1)
sns.scatterplot(y = healthy_glu.index, x = healthy_glu.values, s = healthy_glu.values, color = '#08306b', ax= ax1, alpha = 1)

ax1.axes.get_xaxis().set_visible(False)
ax1.set_xlim(xmin = -500, xmax = 3000)
ax1.set_ylim(ymin = -1.5,ymax = 4.5)

ax1.set_yticklabels( labels = ['Low', 'Normal', 'High', 'Very High'],fontdict = {'font':'times new roman', 'fontsize':16,'fontweight':'bold', 'color':'black'})

ax1.text(-1000,4.3, 'How Glucose level Impact on Having Strokes?' ,{'font': 'times new roman', 'Size': '25','weight':'bold', 'color':'black'})
ax1.text(1700,3.5, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
ax1.text(2050,3.5, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
ax1.text(2075,3.5, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})
ax1.text(-1000,3.8, 'Glucose has no substantial influence on strokes, \nand it is unknown which groups are affected by strokes.', 
        {'font':'Serif', 'size':'16','color': 'black'})

ax1.text(stroke_glu.values[0] + 30,0.05, stroke_glu.values[0], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_glu.values[0] + -355,0.05, healthy_glu.values[0], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})

ax1.text(stroke_glu.values[2] + 30,1.05, stroke_glu.values[2], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_glu.values[2] + 1170,1.05, healthy_glu.values[2], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})

ax1.text(stroke_glu.values[1] + 30,2.05, stroke_glu.values[1], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_glu.values[1] - 1450,2.05, healthy_glu.values[1], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})



# distribution plots ---- only single variable

sns.kdeplot(data = stroke_df, x = 'avg_glucose_level', ax = ax2, shade = True, color = '#1434A4', alpha = 1, )
ax2.set_xlabel('Average Glucose Level', fontdict = {'font':'times new roman', 'color': 'black', 'size': 16,'weight':'bold' })
ax2.text(25,0.025,'Overall Glucose Distribution - How skewed is it?', {'font':'times new roman', 'color': 'black','weight':'bold','size':24})
ax2.text(25,0.021, 'Average glucose levels shows that most of the people have \ncontroled glucose levels.', 
        {'font':'times new roman', 'size':'16','color': 'black'})
ax2.text(210,0.020, 'Total',{'font':'times new roman', 'size':'14','color': '#08306b','weight':'bold'})
ax2.text(240,0.02, '=',{'font':'times new roman', 'size':'14','color': 'black','weight':'bold'})
ax2.text(250,0.02, 'Stroke',{'font':'times new roman', 'size':'14','color': '#d0e0f2','weight':'bold'})
ax2.text(280,0.02, '+',{'font':'times new roman', 'size':'14','color': 'black','weight':'bold'})
ax2.text(290,0.02, 'Healthy',{'font':'times new roman', 'size':'14','color': '#08306b','weight':'bold'})



# distribution plots with hue of strokes


sns.kdeplot(data = stroke_df[stroke_df['stroke'] == 0], x = 'avg_glucose_level',ax = ax3, shade = True,  alpha = 1, color = '#08306b' )
sns.kdeplot(data = stroke_df[stroke_df['stroke'] == 1], x = 'avg_glucose_level',ax = ax3, shade = True,  alpha = 0.8, color = '#d0e0f2')

ax3.set_xlabel('Average Glucose Level', fontdict = {'font':'times new roman', 'color': 'black', 'weight':'bold','size': 16})

ax3.text(-17,0.0195,'Glucose-Stroke Distribution - How serious is it?', {'font':'times new roman', 'weight':'bold','color': 'black', 'size':24})
ax3.text(-17,0.0176,'It is not clear which group of people \neffected by glucose levels.', {'font':'Serif', 'color': 'black', 'size':14})
ax3.text(240,0.0174, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
ax3.text(290,0.0174, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
ax3.text(300,0.0174, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})


fig.text(0.2,1.07,"The Sweet Heart's Tale: Heart Attacks and Blood Sugar Levels",{'font':'times new roman', 'weight':'bold','color': 'black', 'size':35})

fig.show()


In [ ]:
fig = plt.figure(figsize = (24,10),dpi = 60)

gs = fig.add_gridspec(10,24)
gs.update(wspace = 1, hspace = 0.05)


ax2 = fig.add_subplot(gs[1:4,0:8]) #distribution plot
ax3 = fig.add_subplot(gs[6:9, 0:8]) #hue distribution plot
ax1 = fig.add_subplot(gs[2:9,13:]) #dumbbell plot

# axes list
axes = [ ax1,ax2, ax3]

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_yaxis().set_visible(False)
    ax.set_facecolor('#f6f5f5')
    
    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

fig.patch.set_facecolor('#f6f5f5')
        
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(True)
ax1.set_xlim(xmin = -250,xmax = 2000)
ax1.set_ylim(ymin = -1,ymax =3.5)

# dumbbell plot of stoke and healthy people

stroke_bmi = stroke_df[stroke_df['stroke'] == 1].bmi_cat.value_counts()
healthy_bmi = stroke_df[stroke_df['stroke'] == 0].bmi_cat.value_counts()

ax1.hlines(y = ['Obesity', 'Overweight', 'Ideal', 'Underweight'], xmin = [96,115,37,1], 
          xmax = [1797,1495,1159,410], color = 'grey',**{'linewidth':0.5})


sns.scatterplot(y = stroke_bmi.index, x = stroke_bmi.values, s = stroke_bmi.values*2, color = '#d0e0f2', ax= ax1, alpha = 1)
sns.scatterplot(y = healthy_bmi.index, x = healthy_bmi.values, s = healthy_bmi.values*2, color = '#08306b', ax= ax1, alpha = 1)

ax1.set_yticklabels( labels = ['Obesity', 'Overweight', 'Ideal', 'Underweight'],fontdict = {'font':'times new roman', 'fontsize':16,'fontweight':'bold', 'color':'black'})

ax1.text(-750,-1.5, 'How BMI Impact on Having Strokes?' ,{'font': 'times new roman', 'Size': '25','weight':'bold', 'color':'black'})
ax1.text(1000,-1., 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
ax1.text(1250,-1, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
ax1.text(1300,-1, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})
ax1.text(-750,-0.8, 'Signs of probable strokes are indicated by a high BMI, and it is evident \nthat the incidence of strokes is highest among the overweight and obese, \nwhereas it is minimal among younger individuals.', 
        {'font':'times new roman', 'size':'16','color': 'black'})



ax1.text(stroke_bmi.values[0] + 20 , 0.98, stroke_bmi.values[0], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_bmi.values[1] - 275 ,0.98, healthy_bmi.values[1], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})

ax1.text(stroke_bmi.values[1] + 30,0, stroke_bmi.values[1], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#d0e0f2'})
ax1.text(healthy_bmi.values[0] - 300,0, healthy_bmi.values[0], {'font':'times new roman', 'Size':14, 'weight':'bold', 'color':'#08306b'})




# distribution plots ---- only single variable

sns.kdeplot(data = stroke_df, x = 'bmi', ax = ax2, shade = True, color = '#1434A4', alpha = 1, )
ax2.set_xlabel('Body mass index of a person', fontdict = {'font':'times new roman', 'color': 'black', 'size': 16,'weight':'bold' })
ax2.text(-17,0.085,'Overall BMI Distribution - How skewed is it?', {'font':'times new roman', 'color': 'black','weight':'bold','size':24})
ax2.text(-17,0.075, 'BMI is highly skewed towards left side, and averages bmi is around 30.', 
        {'font':'times new roman', 'size':'16','color': 'black'})
ax2.text(80,0.06, 'Total',{'font':'times new roman', 'size':'14','color': '#1434A4','weight':'bold'})
ax2.text(92,0.06, '=',{'font':'times new roman', 'size':'14','color': 'black','weight':'bold'})
ax2.text(97,0.06, 'Stroke',{'font':'times new roman', 'size':'14','color': '#d0e0f2','weight':'bold'})
ax2.text(113,0.06, '+',{'font':'Serif', 'size':'14','color': 'black','weight':'bold'})
ax2.text(117,0.06, 'Healthy',{'font':'times new roman', 'size':'14','color': '#08306b','weight':'bold'})


# distribution plots with hue of strokes


sns.kdeplot(data = stroke_df[stroke_df['stroke'] == 0], x = 'bmi',ax = ax3, shade = True,  alpha = 1, color = '#08306b' )
sns.kdeplot(data = stroke_df[stroke_df['stroke'] == 1], x = 'bmi',ax = ax3, shade = True,  alpha = 0.8, color = '#d0e0f2')

ax3.set_xlabel('Body mass index of a person', fontdict = {'font':'Serif', 'color': 'black', 'weight':'bold','size': 16})

ax3.text(-15,0.12,'BMI-Stroke Distribution - How serious is it?', {'font':'times new roman', 'weight':'bold','color': 'black', 'size':24})
ax3.text(-15,0.11,'Higher BMI higher chance of stroke.', {'font':'times new roman', 'color': 'black', 'size':16})
ax3.text(80,0.095, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
ax3.text(95,0.095, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
ax3.text(97,0.095, 'Healthy', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'})

fig.text(0.25,0.925,'Weight and Heart Attacks: A Tale of a Heavy Heart',{'font':'times new roman', 'weight':'bold','color': 'black', 'size':35})

fig.show()


In [ ]:
#2.2 Overview of univariate categorical variables

In [ ]:
fig = plt.figure(figsize = (15,15),dpi = 40)

gs = fig.add_gridspec(3,3)
gs.update(wspace = 0.2, hspace = 0.5)


ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1:])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])
ax6 = fig.add_subplot(gs[2,0:2])
ax7 = fig.add_subplot(gs[2,2])

axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7]

fig.patch.set_facecolor('#f5f5f5')

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_yaxis().set_visible(False)
    ax.set_facecolor('#f8f8f8')
    ax.spines['bottom'].set_linewidth(2)
    for loc in ['left', 'right', 'top']:
        ax.spines[loc].set_visible(False)
        ax.spines[loc].set_linewidth(2)


title_args = {'font':'times new roman', 'weight':'bold','color': 'black', 'size':24}
font_dict = {'size':16, 'family':'times new roman', 'color':'black', 'weight':'bold'}
health_dict = {'font':'times new roman', 'color': '#08306b', 'size':15, 'weight':'bold'}
dash_dict = {'font':'times new roman', 'color': 'black', 'size':15,'weight':'bold'}
stroke_dict = {'font':'times new roman', 'color': '#d0e0f2', 'size':15,'weight':'bold'}

stroke_col = '#d0e0f2'
healthy_col = '#08306b'

# Ax1: Gender- stroke distributions
healthy_gen = stroke_df[stroke_df['stroke'] == 0].gender.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].gender.value_counts()

ax1.barh( stroke_gen.index , width = healthy_gen.values[0:2], height = 0.2, color = healthy_col)
ax1.barh( np.arange(len(stroke_gen.index)) , width = stroke_gen.values, height = 0.5, color = stroke_col)
ax1.set_yticklabels(stroke_gen.index, **font_dict)

ax1.axes.get_yaxis().set_visible(True)
ax1.axes.get_xaxis().set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(True)
ax1.text(0,1.5, 'Gender Risk',**title_args)
ax1.text(0,1.35, 'Healthy',**health_dict)
ax1.text(790,1.35, '|',**dash_dict)
ax1.text(870,1.35, 'Stroke',**stroke_dict)

# Ax2: work type - stroke distributions
healthy_gen = stroke_df[stroke_df['stroke'] == 0].work_type.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].work_type.value_counts()

ax2.bar( healthy_gen.index , height = healthy_gen.values, width = 0.2, color = healthy_col)
ax2.bar( np.arange(len(stroke_gen.index)) , height = stroke_gen.values, width = 0.5, color= stroke_col)
ax2.set_xticklabels(['Private','Self-Employed','Children', 'Gov-Job','Never worked'], **font_dict)


ax2.text(-0.45,3200, 'Employment Risk',**title_args)
ax2.text(-0.45,2950, 'Healthy',**health_dict)
ax2.text(0.18,2950, '|',**dash_dict)
ax2.text(0.25,2950, 'Stroke',**stroke_dict)

# Ax3: hypertension - stroke distributions

healthy_gen = stroke_df[stroke_df['stroke'] == 0].hypertension.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].hypertension.value_counts()

ax3.bar(['Yes','No'] , height = healthy_gen.values, width = 0.2,color = healthy_col)
ax3.bar( stroke_gen.index, height = stroke_gen.values, width = 0.5,color= stroke_col)
ax3.set_xticklabels(['Yes','No'], **font_dict)

ax3.text(-0.3,5000, 'Hypertension Risk',**title_args)
ax3.text(-0.3,4700, 'Healthy',**health_dict)
ax3.text(0.14,4700, '|',**dash_dict)
ax3.text(0.18,4700, 'Stroke',**stroke_dict)


# Ax4: Heart Disease - stroke distributions

healthy_gen = stroke_df[stroke_df['stroke'] == 0].heart_disease.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].heart_disease.value_counts()

ax4.bar(['Yes','No'] , height = healthy_gen.values, width = 0.2,color = healthy_col)
ax4.bar( stroke_gen.index, height = stroke_gen.values, width = 0.5,color= stroke_col)
ax4.set_xticklabels(['Yes', 'No'],**font_dict)

ax4.text(-0.3,5250, 'Heart Disease Risk',**title_args)
ax4.text(-0.3,4950, 'Healthy',**health_dict)
ax4.text(0.15,4950, '|',**dash_dict)
ax4.text(0.20,4950, 'Stroke',**stroke_dict)


# Ax5: Married - stroke distributions
healthy_gen = stroke_df[stroke_df['stroke'] == 0].ever_married.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].ever_married.value_counts()

ax5.bar( healthy_gen.index , height = healthy_gen.values, width = 0.2,color = healthy_col)
ax5.bar( np.arange(len(stroke_gen.index)) , height = stroke_gen.values, width = 0.5,color= stroke_col )
ax5.set_xticklabels(healthy_gen.index, **font_dict)

ax5.text(-0.3,3500, 'Marrital Status And Risk',**title_args)
ax5.text(-0.3,3300, 'Healthy',**health_dict)
ax5.text(0.14,3300, '|',**dash_dict)
ax5.text(0.18,3300, 'Stroke',**stroke_dict)




# Ax6: Smoking status - stroke distributions

healthy_gen = stroke_df[stroke_df['stroke'] == 0].smoking_status.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].smoking_status.value_counts()

ax6.bar( healthy_gen.index, height = healthy_gen.values, width = 0.2,color = healthy_col)
ax6.bar( np.arange(len(stroke_gen.index)) , height = stroke_gen.values, width = 0.5,color= stroke_col)
ax6.set_xticklabels(['Never Smoked', 'Unknown','Formaly Smoked' ,'Smokes'], **font_dict)

ax6.text(-0.4,2050, 'Smoking Status And Risk',**title_args)
ax6.text(-0.4,1900, 'Healthy',**health_dict)
ax6.text(0.095,1900, '|',**dash_dict)
ax6.text(0.18,1900, 'Stroke',**stroke_dict)




# Ax7: Residence type - stroke distributions

healthy_gen = stroke_df[stroke_df['stroke'] == 0].Residence_type.value_counts()
stroke_gen = stroke_df[stroke_df['stroke'] == 1].Residence_type.value_counts()

ax7.bar( healthy_gen.index , height = healthy_gen.values, width = 0.2,color = healthy_col)
ax7.bar( np.arange(len(stroke_gen.index)) , height = stroke_gen.values, width = 0.5,color= stroke_col)
ax7.set_xticklabels(healthy_gen.index, **font_dict)

ax7.text(-0.31,2800, 'Residence Type And Risk',**title_args)
ax7.text(-0.31,2600, 'Healthy',**health_dict)
ax7.text(0.12,2600,'|',**dash_dict)
ax7.text(0.165,2600, 'Stroke',**stroke_dict)

fig.text(0.05,1.025, 'Univariate Categorical Characteristics: Stroke vs Healthy', {'font':'Serif', 'color':'black','size':30, 'weight':'bold'})
fig.text(0.05,0.9375,'Sometimes, data might be deceptive. All the graphs demonstrate that \nsome characteristics have more strokes than \nothers and are thus more significant, but is this true? \nCounting objectives disregards the overall picture.',{'font':'Serif', 'color':'black','size':20, 'weight':'normal'}, alpha = 0.8)

fig.show()

In [ ]:
##2.3 Univariate analysis of categorical variables

In [ ]:
#Gender Distribution

In [ ]:
#Gender Distribution


stroke_gen = stroke_df[stroke_df['stroke'] == 1]['gender'].value_counts()
healthy_gen = stroke_df[stroke_df['stroke'] == 0]['gender'].value_counts()

female = stroke_df['gender'].value_counts().values[0]
male =  stroke_df['gender'].value_counts().values[1]

stroke_female = int(round (stroke_gen.values[0] / female * 100, 0))
stroke_male = int(round( stroke_gen.values[1] / male *100, 0))
healthy_female = int(round(healthy_gen.values[0] / female * 100, 0))
healthy_male = int(round(healthy_gen.values[1] / male *100, 0))

female_per = int(round(female/(female+male) * 100, 0))
male_per = int(round(male/(female+male)* 100, 0))



fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (7,7),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [healthy_male,stroke_male],
                            'colors' : ['#08306b','#d0e0f2'],
                              'vertical' : True,
                              'interval_ratio_y': 0.1,
                              'interval_ratio_x': 0.1,
                              'icons' : 'male',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.1
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[healthy_female,stroke_female],         
                              'colors' : ['#08306b','#d0e0f2'],
                              'vertical': True,
                              'interval_ratio_y': 0.1,
                              'interval_ratio_x': 0.1,
                              'icons' : 'female',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.1
                                                      
                           }
                         },
                   
)


fig.text(0., 0.8, 'Gender Risk for Stroke - What is the gender influence on strokes?', {'font':'times new roman', 'size':20, 'color':'black', 'weight':'bold'})
fig.text(0., 0.73, 'Males and females have the same probability of suffering a stroke; \nthis disproves our earlier supposition. ', {'font':'times new roman', 'size':13, 'color':'black', 'weight':'normal'}, alpha = 0.7)
fig.text(0.24, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold' ,'color':'#f6f5f5'})
fig.text(0.65, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold', 'color':'#f6f5f5'})
fig.text(0.23, 0.28, '{}%'.format(healthy_male), {'font':'times new roman', 'size':20,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.65, 0.28, '{}%'.format(healthy_female), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.21, 0.67, 'Male ({}%)'.format(male_per), {'font':'times new roman', 'size':14,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.61, 0.67, 'Female({}%)'.format(female_per), {'font':'times new roman', 'size':14,'weight':'bold', 'color':'black'}, alpha = 0.5)

fig.text(0.9,0.73, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(1.02,0.73, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(1.035,0.73, 'No Stroke', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()


In [ ]:
#HyperTension Distribution

stroke_hyper = stroke_df[stroke_df['stroke'] == 1]['hypertension'].value_counts()
healthy_hyper = stroke_df[stroke_df['stroke'] == 0]['hypertension'].value_counts()

no = stroke_df['hypertension'].value_counts().values[0]
yes =  stroke_df['hypertension'].value_counts().values[1]

stroke_no = int(round (stroke_hyper.values[0] / no * 100, 0))
stroke_yes = int(round( stroke_hyper.values[1] / yes *100, 0))
healthy_no = int(round(healthy_hyper.values[0] / no * 100, 0))
healthy_yes = int(round(healthy_hyper.values[1] / yes *100, 0))

no_per = int(round(no/(no+yes) * 100, 0))
yes_per = int(round(yes/(no+yes)* 100, 0))




fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (7,7),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_yes,healthy_yes],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'heartbeat',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':1,
                              'starting_location': 'NE'
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_no,healthy_no],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'heartbeat',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':1,
                              'starting_location': 'NE'
                                                      
                           }
                         },
                   
)


fig.text(0., 0.85, 'Effect of blood pressure on Hypertension and Stroke Risk', {'font':'times new roman', 'size':20, 'color':'black', 'weight':'bold'})
fig.text(0., 0.75, 'Hypertensive individuals are about 10 percent more likely \nto get a stroke than those without hypertension. ', {'font':'Serif', 'size':13, 'color':'black', 'weight':'normal'},alpha = 0.8)
fig.text(0.24, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold' ,'color':'#f6f5f5'})
fig.text(0.65, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold', 'color':'#f6f5f5'})
fig.text(0.23, 0.28, '{}%'.format(healthy_yes), {'font':'times new roman', 'size':20,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.63, 0.28, '{}%'.format(healthy_no), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.1, 0.68, 'Have Hypertension ({}%)'.format(yes_per), {'font':'times new roman', 'size':14,'weight':'bold' ,'color':'black'},alpha = 0.7,)
fig.text(0.55, 0.68, "Don't have Hypertension({}%)".format(no_per), {'font':'times new roman', 'size':14,'weight':'bold', 'color':'black'}, alpha = 0.7)

fig.text(0.90,0.75, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(1.02,0.75, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(1.04,0.75, 'No Stroke', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()

In [ ]:
#HeartDisease Distribution


stroke_hyper = stroke_df[stroke_df['stroke'] == 1]['heart_disease'].value_counts()
healthy_hyper = stroke_df[stroke_df['stroke'] == 0]['heart_disease'].value_counts()

no = stroke_df['heart_disease'].value_counts().values[0]
yes =  stroke_df['heart_disease'].value_counts().values[1]

stroke_no = int(round (stroke_hyper.values[0] / no * 100, 0))
stroke_yes = int(round( stroke_hyper.values[1] / yes *100, 0))
healthy_no = int(round(healthy_hyper.values[0] / no * 100, 0))
healthy_yes = int(round(healthy_hyper.values[1] / yes *100, 0))

no_per = int(round(no/(no+yes) * 100, 0))
yes_per = int(round(yes/(no+yes)* 100, 0))




fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (7,7),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_yes,healthy_yes],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'heart',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_no,healthy_no],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'heart',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                         },
                   
)


fig.text(0., 0.85, 'Heart disease and Stroke Risk - an impact of Heart disease', {'font':'times new roman', 'size':20, 'color':'black', 'weight':'bold'})
fig.text(0., 0.75, 'Nearly 12 percent of those who have had a cardiac issue \nin the past suffer from a stroke. ', {'font':'Serif', 'size':13, 'color':'black', 'weight':'normal'}, alpha = 0.8)
fig.text(0.24, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold' ,'color':'#f6f5f5'})
fig.text(0.65, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold', 'color':'#f6f5f5'})
fig.text(0.25, 0.27, '{}%'.format(healthy_yes), {'font':'times new roman', 'size':20,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.65, 0.27, '{}%'.format(healthy_no), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.12, 0.68, 'UnHealthy Heart ({}%)'.format(yes_per), {'font':'times new roman', 'size':16,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.55, 0.68, "Healthy Heart({}%)".format(no_per), {'font':'times new roman', 'size':16,'weight':'bold', 'color':'black'}, alpha = 0.5)

fig.text(0.9,0.75, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(1.02,0.75, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(1.04,0.75, 'No Stroke', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()

In [ ]:
#Marriage Distribution

stroke_mary = stroke_df[stroke_df['stroke'] == 1]['ever_married'].value_counts()
healthy_mary = stroke_df[stroke_df['stroke'] == 0]['ever_married'].value_counts()

yes = stroke_df['ever_married'].value_counts().values[0]
no =  stroke_df['ever_married'].value_counts().values[1]

stroke_no = int(round (stroke_mary.values[1] / no * 100, 0))
stroke_yes = int(round( stroke_mary.values[0] / yes *100, 0))
healthy_no = int(round(healthy_mary.values[1] / no * 100, 0))
healthy_yes = int(round(healthy_mary.values[0] / yes *100, 0))

no_per = int(round(no/(no+yes) * 100, 0))
yes_per = int(round(yes/(no+yes)* 100, 0))




fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (7,7),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_yes,healthy_yes],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'ring',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_no,healthy_no],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'universal-access',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                         },
                   
)


fig.text(0., 0.8, 'What are the impacts of marriage on the heart?', {'font':'times new roman', 'size':20, 'color':'black', 'weight':'bold'})
fig.text(0., 0.74, 'The risk of having a stroke among married persons is relative, \nhigh, and only differs by 5 percent.', {'font':'Serif', 'size':13, 'color':'black', 'weight':'normal'}, alpha = 0.8)
fig.text(0.24, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold' ,'color':'#f6f5f5'})
fig.text(0.65, 0.22, 'ooo', {'font':'times new roman', 'size':16,'weight':'bold', 'color':'#f6f5f5'})
fig.text(0.25, 0.28, '{}%'.format(healthy_yes), {'font':'times new roman', 'size':20,'weight':'bold' , 'color':'#08306b'}, alpha = 1,)
fig.text(0.65, 0.28, '{}%'.format(healthy_no), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.20, 0.68, 'Married({}%)'.format(yes_per), {'font':'times new roman', 'size':16,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.58, 0.68, "Unmarried({}%)".format(no_per), {'font':'times new roman', 'size':16,'weight':'bold', 'color':'black'}, alpha = 0.5)

fig.text(0.9,0.72, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(1.02,0.72, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(1.04,0.72, 'No Stroke', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()

In [ ]:
#Residence Distribution


stroke_home = stroke_df[stroke_df['stroke'] == 1]['Residence_type'].value_counts()
healthy_home= stroke_df[stroke_df['stroke'] == 0]['Residence_type'].value_counts()

urban = stroke_df['Residence_type'].value_counts().values[0]
rural =  stroke_df['Residence_type'].value_counts().values[1]

stroke_urban = int(round (stroke_home.values[0] / urban * 100, 0))
stroke_rural= int(round( stroke_home.values[1] / rural *100, 0))
healthy_urban = int(round(healthy_home.values[0] / urban * 100, 0))
healthy_rural = int(round(healthy_home.values[1] / rural *100, 0))

urban_per = int(round(urban/(urban+rural) * 100, 0))
rural_per = int(round(rural/(urban+rural)* 100, 0))




fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (7,7),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_urban,healthy_urban],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'city',
                              'icon_legend': False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_rural,healthy_rural],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'home',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                         },
                   
)


fig.text(0., 0.85, 'Lifestyle and Strokes: What influence does geography have on strokes?', {'font':'Serif', 'size':20, 'color':'black', 'weight':'bold'})
fig.text(0., 0.79, "The location of a person's residence has little influence on heart attacks. \nBoth rural and urban populations are equally susceptible to Strokes.", {'font':'Serif', 'size':13, 'color':'black', 'weight':'normal'}, alpha =0.8)

fig.text(0.23, 0.28, '{}%'.format(healthy_urban), {'font':'Serif', 'size':20,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.68, 0.28, '{}%'.format(healthy_rural), {'font':'Serif', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.13, 0.68, 'Urban Home({}%)'.format(urban_per), {'font':'Serif', 'size':16,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.57, 0.68, "Rural Home({}%)".format(rural_per), {'font':'Serif', 'size':16,'weight':'bold', 'color':'black'}, alpha = 0.5)
#fig.text(0., 0.8, 'Assumption was proven wrong', {'font':'Serif', 'size':24, 'color':'black', 'weight':'bold'})

fig.text(0.88,0.75, 'Stroke ', {'font': 'Serif','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(1,0.75, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(1.025,0.75, 'No Stroke', {'font': 'Serif','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()

In [ ]:
#Smoking Distribution

smoke = stroke_df['smoking_status'].value_counts()
stroke_smoke = stroke_df[stroke_df['stroke'] == 1]['smoking_status'].value_counts()
healthy_smoke = stroke_df[stroke_df['stroke'] == 0]['smoking_status'].value_counts()

never = smoke.values[0]
unknown =  smoke.values[1]
former = smoke.values[2]
smokes = smoke.values[3]

stroke_never = int(round (stroke_smoke.values[0] / never * 100, 0))
stroke_unknown = int(round( stroke_smoke.values[2]  / unknown *100, 0))
stroke_former = int(round (stroke_smoke.values[1]  / former * 100, 0))
stroke_smokes = int(round( stroke_smoke.values[3]  / smokes *100, 0))

healthy_never = int(round(healthy_smoke.values[0] / never * 100, 0))
healthy_unknown = int(round(healthy_smoke.values[1] / unknown *100, 0))
healthy_former = int(round(healthy_smoke.values[2] / former * 100, 0))
healthy_smokes = int(round(healthy_smoke.values[3]/ smokes *100, 0))


never_per = int(round(never/(never+unknown+former+smokes) * 100, 0))
unknown_per = int(round(unknown/(never+unknown+former+smokes)* 100, 0))
former_per = int(round(former/(never+unknown+former+smokes) * 100, 0))
smokes_per = int(round(smokes/(never+unknown+former+smokes)* 100, 0))



fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (15,15),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'141':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_never,healthy_never],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'ban',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '142' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_former,healthy_former],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'smoking-ban',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           },
                          
                          '143':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_unknown,healthy_unknown],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'question-circle',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '144' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_smokes,healthy_smokes],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.006,
                              'interval_ratio_y':0.006,
                              'icons' : 'smoking',
                              'icon_legend' :False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                          
                         },
                   
)


fig.text(0.1, 0.65, 'Smoking and Stroke - Can a smoking habit lead to a Stroke?', {'font':'times new roman', 'size':20, 'color':'black', 'weight':'bold'})
fig.text(0.1, 0.62, 'The risk of stroke associated with smoking is intriguing, as it appears smoking does have an influence on strokes, \nand former smokers are the most likely to get a stroke. ', {'font':'Serif', 'size':13, 'color':'black', 'weight':'normal'}, alpha = 0.7)
fig.text(0.18, 0.38, '{}%'.format(healthy_never), {'font':'times new roman', 'size':24,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.38, 0.38, '{}%'.format(healthy_former), {'font':'times new roman', 'size':24,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.58, 0.38, '{}%'.format(healthy_unknown), {'font':'times new roman', 'size':24,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.78, 0.38, '{}%'.format(healthy_smokes), {'font':'times new roman', 'size':24,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.14, 0.58, 'Never Smoked({}%)'.format(never_per), {'font':'times new roman', 'size':14,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.33, 0.58, "Formerly Smoked({}%)".format(former_per), {'font':'times new roman', 'size':14,'weight':'bold', 'color':'black'}, alpha = 0.5)
fig.text(0.55, 0.58, 'Unknown({}%)'.format(unknown_per), {'font':'times new roman', 'size':14,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.75, 0.58, "Smokes({}%)".format(smokes_per), {'font':'times new roman', 'size':14,'weight':'bold', 'color':'black'}, alpha = 0.5)

fig.text(0.7,0.62, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(0.76,0.62, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(0.77,0.62, 'No Stroke', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()


In [ ]:
#Work Distribution


work = stroke_df['work_type'].value_counts()
stroke_work = stroke_df[stroke_df['stroke'] == 1]['work_type'].value_counts()
healthy_work = stroke_df[stroke_df['stroke'] == 0]['work_type'].value_counts()

private = work.values[0]
self =  work.values[1]
child = work.values[2]
gov = work.values[3]
never = work.values[4]

stroke_private = int(round (stroke_work.values[0] / private * 100, 0))
stroke_self = int(round( stroke_work.values[1]  / self *100, 0))
stroke_child = int(round (stroke_work.values[3]  / child * 100, 0))
stroke_gov = int(round( stroke_work.values[2]  / gov *100, 0))
stroke_never = int(round( 0, 0))

healthy_private = int(round(healthy_work.values[0] / private * 100, 0))
healthy_self = int(round(healthy_work.values[1] / self *100, 0))
healthy_child = int(round(healthy_work.values[2] / child * 100, 0))
healthy_gov = int(round(healthy_work.values[3]/ gov *100, 0))
healthy_never = int(round(healthy_work.values[4]/ never *100, 0))

private_per = int(round(private/(private+self+child+gov+never) * 100, 0))
self_per = int(round(self/(private+self+child+gov+never)* 100, 0))
child_per = int(round(child/(private+self+child+gov+never) * 100, 0))
gov_per = int(round(gov/(private+self+child+gov+never)* 100, 0))
never_per = int(round(never/(private+self+child+gov+never)* 100, 0))


fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (15,15),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'151':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_private,healthy_private],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'circle',
                              'icon_legend': False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.2,
                              'starting_location': 'NE'
                          },
                          
                          '152' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_self,healthy_self],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'circle',
                              'icon_legend' :False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.2,
                              'starting_location': 'NE'
                                                      
                           },
                          
                          '153':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [stroke_gov,healthy_gov],
                            'colors' : ['#d0e0f2','#08306b'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'circle',
                              'icon_legend': False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.2,
                              'starting_location': 'NE'
                          },
                          
                          '154' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_never,healthy_never],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.006,
                              'interval_ratio_y':0.006,
                              'icons' : 'circle',
                              'icon_legend' :False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.2,
                              'starting_location': 'NE'
                                                      
                           },
                          
                          '155' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[stroke_child,healthy_child],         
                              'colors' : ['#d0e0f2','#08306b'],
                              'vertical': True,
                              'interval_ratio_x': 0.006,
                              'interval_ratio_y':0.006,
                              'icons' : 'circle',
                              'icon_legend' :False,
                              'icon_size':15,
                              'plot_anchor':'C',
                              'alpha':0.2,
                              'starting_location': 'NE'
                                                      
                           }
                          
                         },
                   
)


fig.text(0.1, 0.65, 'Work and Stroke - Does occupational stress cause Stroke?', {'font':'times new roman', 'size':20, 'color':'black', 'weight':'bold'},alpha = 0.9)
fig.text(0.1, 0.62, 'According to percentages, self-employed individuals are more likely to experience a stroke, \nbut the majority of strokes might be observed among privately employed individuals, and could be a result of job stress.', {'font':'Serif', 'size':13, 'color':'black', 'weight':'normal'}, alpha = 0.7)
fig.text(0.16, 0.40, '{}%'.format(healthy_private), {'font':'times new roman', 'size':20,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.34, 0.40, '{}%'.format(healthy_self), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.48, 0.40, '{}%'.format(healthy_gov), {'font':'times new roman', 'size':20,'weight':'bold' ,'color':'#08306b'},alpha = 1,)
fig.text(0.64, 0.40, '{}%'.format(healthy_never), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)
fig.text(0.8, 0.40, '{}%'.format(healthy_child), {'font':'times new roman', 'size':20,'weight':'bold', 'color':'#08306b'}, alpha = 1)


fig.text(0.15, 0.57, 'Private({}%)'.format(private_per), {'font':'times new roman', 'size':13,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.33, 0.57, "Self({}%)".format(self_per), {'font':'times new roman', 'size':13,'weight':'bold', 'color':'black'}, alpha = 0.5)
fig.text(0.46, 0.57, 'Government{}%)'.format(gov_per), {'font':'times new roman', 'size':13,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.61, 0.57, "Never worked({}%)".format(never_per), {'font':'times new roman', 'size':13,'weight':'bold', 'color':'black'}, alpha = 0.5)
fig.text(0.78, 0.57, "Children({}%)".format(child_per), {'font':'times new roman', 'size':13,'weight':'bold', 'color':'black'}, alpha = 0.5)         

fig.text(0.7,0.6, 'Stroke ', {'font': 'times new roman','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#d0e0f2'})
fig.text(0.76,0.6, '|', {'color':'black' , 'size':'16', 'weight': 'bold'})
fig.text(0.77,0.6, 'No Stroke', {'font': 'times new roman','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#08306b'},alpha = 1)


fig.show()

In [ ]:
##2.4 Relationship between two continuous variables

In [ ]:
#Distribution of Strokes with Gender & Marriage

import matplotlib

fig = plt.figure(figsize=(12,6),dpi = 100)
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.25, hspace=0.5)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = stroke_df[stroke_df['stroke']==0]
stroke = stroke_df[stroke_df['stroke']==1]


col1 = ["#4b4b4c","#d0e0f2"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#08306b"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

stroke = pd.crosstab(stroke['gender'],[stroke['ever_married']],normalize='index')
no_stroke = pd.crosstab(healthy['gender'],[healthy['ever_married']], normalize='index')

sns.heatmap(ax=ax0, data=stroke, linewidths= 0,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_stroke[0:-1], linewidths=0, 
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0, -0.69, 'Stroke Distribution in Relation to Gender and Marriage', {'font':'times new roman', 'color':'black', 'weight':'bold','size':25})
ax0.text(0, -0.34, 'It is evident that married individuals have a higher incidence of strokes than singles. \nMarried men are the most influential, followed by married women.', {'font':'Serif', 'color':'black','size':14}, alpha = 0.7)

ax0.text(0,-0.1,'Storke Pecentage ', {'font':'serif', 'color':"#d0e0f2", 'size':20},alpha = 0.9)
ax1.text(0,-0.1,'No Stroke Percentage', {'font':'serif', 'color':"#08306b", 'size':20}, alpha =0.9)

ax0.axes.set_xticklabels(['Single', 'Married'], {'font':'times new roman', 'color':'black', 'size':16})
ax1.axes.set_xticklabels(['Single', 'Married'], {'font':'times new roman', 'color':'black', 'size':16})

ax0.axes.set_yticklabels(['Female', 'Male'], {'font':'times new roman', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
fig.show()


In [ ]:
#Distribution of Strokes with Gender & Work type

fig = plt.figure(figsize=(12,6), dpi = 100)
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.25, hspace=0.5)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = stroke_df[stroke_df['stroke']==0]
stroke = stroke_df[stroke_df['stroke']==1]

gender_order = ['Female','Male']
work_order = ['Private', 'Self-employed','Govt_job', 'children']


col1 = ["#4b4b4c","#d0e0f2"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#08306b"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

stroke = pd.crosstab(stroke['gender'],[stroke['work_type']],normalize='index').loc[gender_order,work_order]
no_stroke = pd.crosstab(healthy['gender'],[healthy['work_type']], normalize='index').loc[gender_order,work_order]

sns.heatmap(ax=ax0, data=stroke, linewidths= 0,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_stroke, linewidths=0, 
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0, -1., 'Distribution of Strokes by Gender and Occupation', {'font':'times new roman', 'color':'black', 'weight':'bold','size':25})
ax0.text(0, -0.75, 'Strokes are more common among private-sector workers than those engaged in the public sector..', {'font':'times new roman', 'color':'black','size':14}, alpha = 0.7)

ax0.text(0,-0.1,'Storke Pecentage ', {'font':'times new roman', 'color':"#d0e0f2", 'size':20},alpha = 0.9)
ax1.text(0,-0.1,'No Stroke Percentage', {'font':'times new roman', 'color':"#08306b", 'size':20}, alpha =0.9)

ax0.axes.set_xticklabels(['Private', 'Self-Emp','Govt-Job', 'Children'], {'font':'times new roman', 'color':'black', 'size':12})
ax1.axes.set_xticklabels(['Private', 'Self-Emp','Govt-Job', 'Children'], {'font':'times new roman', 'color':'black', 'size':12})

ax0.axes.set_yticklabels(gender_order, {'font':'times new roman', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
fig.show()


In [ ]:
#Distribution of Strokes with Gender & Smoking Status

fig = plt.figure(figsize=(12,6), dpi = 100)
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.25, hspace=0.5)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = stroke_df[stroke_df['stroke']==0]
stroke = stroke_df[stroke_df['stroke']==1]

gender_order = ['Female','Male']
smoking_order = ['smokes', 'formerly smoked', 'Unknown', 'never smoked']


col1 = ["#4b4b4c","#d0e0f2"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#08306b"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

stroke = pd.crosstab(stroke['gender'],[stroke['smoking_status']],normalize='index').loc[gender_order,smoking_order]
no_stroke = pd.crosstab(healthy['gender'],[healthy['smoking_status']], normalize='index').loc[gender_order,smoking_order]

sns.heatmap(ax=ax0, data=stroke, linewidths= 0,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_stroke, linewidths=0, 
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0, -1., 'Distribution of Strokes by Gender and Smoking Status', {'font':'times new roman', 'color':'black', 'weight':'bold','size':25})
ax0.text(0, -0.55, 'Strokes are more prevalent among men who have quit smoking \nand women who have never smoked.', {'font':'Serif', 'color':'black','size':14}, alpha = 0.7)

ax0.text(0,-0.1,'Storke Pecentage ', {'font':'times new roman', 'color':"#d0e0f2", 'size':20},alpha = 0.9)
ax1.text(0,-0.1,'No Stroke Percentage', {'font':'times new roman', 'color':"#08306b", 'size':20}, alpha =0.9)

ax0.axes.set_xticklabels(['Smoke', 'Quit','No Info', 'Never'], {'font':'times new roman', 'color':'black', 'size':12})
ax1.axes.set_xticklabels(['Smoke', 'Quit','No Info', 'Never'], {'font':'times new roman', 'color':'black', 'size':12})

ax0.axes.set_yticklabels(gender_order, {'font':'times new roman', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
fig.show()


In [ ]:
#Distribution of Strokes with Gender & Age

fig = plt.figure(figsize=(12,6), dpi = 100)
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.25, hspace=0.5)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = stroke_df[stroke_df['stroke']==0]
stroke = stroke_df[stroke_df['stroke']==1]

gender_order = ['Female','Male']
age_order = ['Children', 'Teens', 'Mid Adults', 'Elderly']


col1 = ["#4b4b4c","#d0e0f2"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#08306b"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

stroke = pd.crosstab(stroke['gender'],[stroke['age_cat']],normalize='index').loc[gender_order,age_order]
no_stroke = pd.crosstab(healthy['gender'],[healthy['age_cat']], normalize='index').loc[gender_order,age_order]

sns.heatmap(ax=ax0, data=stroke, linewidths= 0,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_stroke, linewidths=0, 
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0, -1., 'Distribution of Stroke by Gender and Age', {'font':'times new roman', 'color':'black', 'weight':'bold','size':25})
ax0.text(0, -0.75, 'Elderly men and women are both susceptible to heart attacks, regardless of gender.', {'font':'times new roman', 'color':'black','size':14}, alpha = 0.7)

ax0.text(0,-0.1,'Storke Pecentage ', {'font':'serif', 'color':"#d0e0f2", 'size':20},alpha = 0.9)
ax1.text(0,-0.1,'No Stroke Percentage', {'font':'serif', 'color':"#08306b", 'size':20}, alpha =0.9)

ax0.axes.set_xticklabels(age_order, {'font':'times new roman', 'color':'black', 'size':12})
ax1.axes.set_xticklabels(age_order, {'font':'times new roman', 'color':'black', 'size':12})

ax0.axes.set_yticklabels(gender_order, {'font':'times new roman', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
fig.show()

In [ ]:
#Distribution of Strokes with Gender & Glucose level


fig = plt.figure(figsize=(12,6), dpi = 100)
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.25, hspace=0.5)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = stroke_df[stroke_df['stroke']==0]
stroke = stroke_df[stroke_df['stroke']==1]

gender_order = ['Female','Male']
glucose_order = ['Low', 'Normal', 'High', 'Very High']


col1 = ["#4b4b4c","#d0e0f2"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#08306b"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

stroke = pd.crosstab(stroke['gender'],[stroke['glucose_cat']],normalize='index').loc[gender_order,glucose_order]
no_stroke = pd.crosstab(healthy['gender'],[healthy['glucose_cat']], normalize='index').loc[gender_order,glucose_order]

sns.heatmap(ax=ax0, data=stroke, linewidths= 0,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_stroke, linewidths=0, 
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0, -1., 'Distribution of Strokes with Gender & Glucose level', {'font':'times new roman', 'color':'black', 'weight':'bold','size':25})
ax0.text(0, -0.75, 'Clearly, irrespective of gender, elderly men and women are prone to heart strokes.', {'font':'Serif', 'color':'black','size':14}, alpha = 0.7)

ax0.text(0,-0.1,'Storke Pecentage ', {'font':'times new roman', 'color':"#d0e0f2", 'size':20},alpha = 0.9)
ax1.text(0,-0.1,'No Stroke Percentage', {'font':'times new roman', 'color':"#08306b", 'size':20}, alpha =0.9)

ax0.axes.set_xticklabels(glucose_order, {'font':'times new roman', 'color':'black', 'size':12})
ax1.axes.set_xticklabels(glucose_order, {'font':'times new roman', 'color':'black', 'size':12})

ax0.axes.set_yticklabels(gender_order, {'font':'times new roman', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
fig.show()


In [ ]:
#Distribution of Strokes with Hypertension & Heart disease

fig = plt.figure(figsize=(12,6))
gs = fig.add_gridspec(1,2)
gs.update(wspace=0.25, hspace=0.5)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = stroke_df[stroke_df['stroke']==0]
stroke = stroke_df[stroke_df['stroke']==1]


col1 = ["#4b4b4c","#d0e0f2"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#08306b"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

stroke = pd.crosstab(stroke['hypertension'],[stroke['heart_disease']],normalize='index')
no_stroke = pd.crosstab(healthy['hypertension'],[healthy['heart_disease']], normalize='index')

sns.heatmap(ax=ax0, data=stroke, linewidths= 0,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_stroke, linewidths=0, 
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0, -0.69, 'Distribution of Strokes with Hypertension & Heart disease', {'font':'times new roman', 'color':'black', 'weight':'bold','size':25})
ax0.text(0, -0.42, 'The majority of strokes occur in individuals without \ncardiac disease but with hypertension.', {'font':'times new roman', 'color':'black','size':14}, alpha = 0.7)

ax0.text(0,-0.1,'Storke Pecentage ', {'font':'times new roman', 'color':"#d0e0f2", 'size':20},alpha = 0.9)
ax1.text(0,-0.1,'No Stroke Percentage', {'font':'times new roman', 'color':"#08306b", 'size':20}, alpha =0.9)

ax0.axes.set_xticklabels(['Well heart', 'Ill heart'], {'font':'times new roman', 'color':'black', 'size':16})
ax1.axes.set_xticklabels(['Well heart', 'Ill heart'], {'font':'times new roman', 'color':'black', 'size':16})

ax0.axes.set_yticklabels(['No hypertension', 'Have hypertension'], {'font':'serif', 'color':'black', 'size':16}, rotation= 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
fig.show()


In [ ]:
##2.5 Visualization of correlations between various features

In [ ]:
#Correlation Map of Features - How closly each of the features correlated?

stroke_df_copy = stroke_df.copy()
# feature log transformations 

stroke_df_copy['age'] = stroke_df_copy['age'].apply(lambda x: np.log(x+10)*3)
stroke_df_copy['avg_glucose_level'] = stroke_df_copy['avg_glucose_level'].apply(lambda x: np.log(x+10)*2)
stroke_df_copy['bmi'] = stroke_df_copy['bmi'].apply(lambda x: np.log(x+10)*2)



# preprocessing - label enconding and numerical value scaling
ohe = OneHotEncoder()
ss = StandardScaler()
le = LabelEncoder()

## label encoding of ordinal categorical features
for col in stroke_df_copy.columns:
    stroke_df_copy[col] = le.fit_transform(stroke_df_copy[col])
    
cols = stroke_df_copy.columns
## normalizing with standard scaler of numerical features
stroke_df_copy[cols] = ss.fit_transform(stroke_df_copy[cols])


# correlation map for all the features
stroke_df_corr = stroke_df_copy.drop(columns = ['id']).corr()
mask = np.triu(np.ones_like(stroke_df_corr, dtype=np.bool))

fig, ax = plt.subplots(figsize = (8,8))
fig.patch.set_facecolor('#f6f5f5')
ax.set_facecolor('#f6f5f5')

mask = mask[1:, :-1]
corr = stroke_df_corr.iloc[1:,:-1].copy()


colors = ['#f6f5f5','#08306b','#d0e0f2']
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

# plot heatmap
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f",cmap = colormap,
           vmin=-0.15, vmax=0.5, cbar_kws={"shrink": .5, }, ax = ax, cbar = False,
           linewidth = 1,linecolor = '#f6f5f5', square = True,annot_kws = {'font':'times new roman', 'size':10, 'color':'black'} )
# yticks
ax.tick_params(axis = 'y', rotation=0)
xticks = ['Gender', 'Age','Hyper tension', 'Heart Disease', 'Marriage', 'Work', 'Residence', 'Glucose Level', 'BMI', 'Smoking Status','Stroke','BMI Cat','Age Cat']
yticks = ['Gender', 'Age','Hyper tension', 'Heart Disease', 'Marriage', 'Work', 'Residence', 'Glucose Level', 'BMI', 'Smoking Status','Stroke','BMI Cat','Age Cat']
ax.set_xticklabels(xticks, {'font':'times new roman', 'size':10, 'weight':'bold'},rotation = 90, alpha = 0.9)
ax.set_yticklabels(yticks, {'font':'times new roman', 'size':10, 'weight':'bold'}, rotation = 0, alpha = 0.9)
ax.text(-3.5,-1.1, 'Feature Correlation Map - How closely are the various characteristics connected to one another?',{'font':'times new roman', 'size': 16, 'weight':'bold'}, alpha = 0.9)
ax.text(-3.5,-0.65, 'A Quick Look at Correlation for Feature Data that has been Processed.',{'font':'times new roman', 'size': 12, 'weight':'normal'}, alpha = 0.8)

ax.text(9,5, 'Among the most strongly connected variables \nare age and marital status \n(which are positively correlated), \nand employment (which is negatively correlated).',{'font':'times new roman', 'size': 11, 'weight':'bold'},alpha = 0.7)
ax.text(9,3.7, 'Insight:',{'font':'times new roman', 'size': 12, 'weight':'bold'},alpha = 0.9)


fig.show()

In [ ]:
#Visualization of Clustering of Each Feature with Other

labels = ['Smoking', 'BMI','Age', 'Marriage', 'Heart Disease', 'Stroke','Hypertension', 'Age Cat', 'Gender', 'Work', 'BMI Cat', 'Residence','Glucose Level', 'Glucose Cat' ]


g = sns.clustermap(stroke_df_corr, annot = True, fmt = '0.2f',
                   cbar= False, cbar_pos=(0,0, 0,0),linewidth = 0.5,
                   cmap = colormap,dendrogram_ratio=0.1,
                   facecolor = '#f6f5f5', figsize = (8,8),square = True,
                   annot_kws = {'font':'times new roman', 'size':10, 'color':'black'} )

plt.gcf().set_facecolor('#f6f5f5')
label_args = {'font':'times new roman', 'font':18, 'weight':'bold'}
plt.setp(g.ax_heatmap.set_yticklabels(labels), rotation=0, fontsize = 10, fontfamily = 'times new roman', fontweight = 'bold', alpha = 0.8)  # For y axis
plt.setp(g.ax_heatmap.set_xticklabels(labels), rotation=90, fontsize = 10, fontfamily = 'times new roman', fontweight = 'bold', alpha = 0.8) # For x axis
g.fig.text(0,1.065,'Visualization of Clustering of Each Feature with Other Features',{'font':'times new roman', 'size':16, 'weight':'bold'})
g.fig.text(0,1.015,'The lines on the top and left of the cluster map that represent \nthe reliance of characteristics are termed dendrograms.',{'font':'times new roman', 'size':12}, alpha = 0.8)
plt.show()

In [ ]:
####3.  STROKE PREDICTION ANALYSIS

In [ ]:
#load dataset
stroke_df= pd.read_csv('/Users/apple/Documents/TU/second semester/Machine learning/STROKE/ICA work/healthcare-dataset-stroke-data.csv', delimiter = ',', encoding = 'utf-8')

In [ ]:
stroke_df.head(5)

In [ ]:
stroke_df.info()

In [ ]:
# Create age groups
stroke_df['age_group'] = 0
for i in range(len(stroke_df.index)):
    if stroke_df.iloc[i, 2] < 2:
        stroke_df.iloc[i, 12] = 'baby'
    elif stroke_df.iloc[i, 2] < 17 and stroke_df.iloc[i, 2] >= 2:
        stroke_df.iloc[i, 12] = 'child'
    elif stroke_df.iloc[i, 2] < 30 and stroke_df.iloc[i, 2] >= 17:
        stroke_df.iloc[i, 12] = 'young adults'
    elif stroke_df.iloc[i, 2] < 60 and stroke_df.iloc[i, 2] >= 30:
        stroke_df.iloc[i, 12] = 'middle-aged adults'
    elif stroke_df.iloc[i, 2] < 80 and stroke_df.iloc[i, 2] >= 60:
        stroke_df.iloc[i, 12] = 'old-aged adults'
    else:
        stroke_df.iloc[i, 12] = 'long-lived'
        
# Imputing missing values in bmi
mean_bmi = stroke_df.groupby(['gender', 'age_group']).agg({'bmi': 'mean'}).reset_index()
for i in range(len(stroke_df.index)):
    if pd.isna(stroke_df.iloc[i, 9]) == True:
        for j in range(len(mean_bmi.index)):
            if mean_bmi.iloc[j, 0] == stroke_df.iloc[i, 1] and mean_bmi.iloc[j, 1] == stroke_df.iloc[i, 12]:
                stroke_df.iloc[i, 9] = mean_bmi.iloc[j, 2]

# Create bmi groups
stroke_df['bmi_group'] = 0
for i in range(len(stroke_df.index)):
    if stroke_df.iloc[i, 9] < 18.5:
        stroke_df.iloc[i, 13] = 'Underweight'
    elif stroke_df.iloc[i, 9] < 25.0 and stroke_df.iloc[i, 9] >= 18.5:
        stroke_df.iloc[i, 13] = 'Normal weight'
    elif stroke_df.iloc[i, 9] < 30.0 and stroke_df.iloc[i, 9] >= 25.0:
        stroke_df.iloc[i, 13] = 'Overweight'
    else:
        stroke_df.iloc[i, 13] = 'Obese'
        
# Create glucose groups
stroke_df['glucose_group'] = 0
for i in range(len(stroke_df.index)):
    if stroke_df.iloc[i, 8] < 100:
        stroke_df.iloc[i, 14] = 'Normal'
    elif stroke_df.iloc[i, 8] >= 100 and stroke_df.iloc[i, 8] < 125:
        stroke_df.iloc[i, 14] = 'Prediabetes'
    else:
        stroke_df.iloc[i, 14] = 'Diabetes'


In [ ]:
# Grouping by categorical features
dst_st_age = stroke_df.groupby(['age_group', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
hyper = stroke_df.groupby(['hypertension', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
heart = stroke_df.groupby(['heart_disease', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
marry = stroke_df.groupby(['ever_married', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
work = stroke_df.groupby(['work_type', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
residence = stroke_df.groupby(['Residence_type', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
glucose_group = stroke_df.groupby(['glucose_group', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
bmi_group = stroke_df.groupby(['bmi_group', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
smoking = stroke_df.query('smoking_status != "Unknown"').groupby(['smoking_status', 'stroke']).agg({'stroke': 'count'})\
.rename(columns = {'stroke': 'count'}).reset_index()

# Create percent column for data frames
def percent(data):
    data['percent'] = 0
    for i in range(len(data.index)):
        if i < len(data.index)-1:
            if data.iloc[i, 0] == data.iloc[i+1, 0]:
                data.iloc[i, 3] = round((data.iloc[i, 2] / (data.iloc[i, 2] + data.iloc[i+1, 2])) * 100, 1)
            elif data.iloc[i, 0] == data.iloc[i-1, 0]:
                data.iloc[i, 3] = 100 - data.iloc[i-1, 3]
            else:
                data.iloc[i, 3] = 100.0
        else:
            if data.iloc[i, 0] == data.iloc[i-1, 0]:
                data.iloc[i, 3] = 100 - data.iloc[i-1, 3]
            else:
                data.iloc[i, 3] = 100.0
            
percent(dst_st_age)
percent(hyper)
percent(heart)
percent(marry)
percent(work)
percent(residence)
percent(glucose_group)
percent(bmi_group)
percent(smoking)

dst_st_age.iloc[[0,2,4,6,8,10], 1] = "Didn't have a stroke"
dst_st_age.iloc[[1,3,5,7,9], 1] = "Had a stroke"

hyper.iloc[[0,1], 0] = 'No hypertension'
hyper.iloc[[2,3], 0] = 'Hypertension'

heart.iloc[[0,1], 0] = 'No heart diseases'
heart.iloc[[2,3], 0] = 'Heart diseases'


In [ ]:
X = stroke_df.drop(['id', 'stroke'], axis = 1)
y = stroke_df['stroke']

num_cols = X.select_dtypes(include = ['int64', 'float64']).columns.to_list()
cat_cols = X.select_dtypes(include = ['object']).columns.to_list()

In [ ]:
def label_encoder(df):
    for i in cat_cols:
        le = LabelEncoder()
        df[i] = le.fit_transform(df[i])
    return df

In [ ]:
sc = StandardScaler()
X[num_cols] = sc.fit_transform(X[num_cols])

# Label encoding
X = label_encoder(X)

X.head()

In [ ]:
stroke_df.info()

In [ ]:
stroke_df['stroke'].value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 22)

smote = SMOTE()

X_train_balanced, Y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
results = pd.DataFrame(columns = ['SVC', 'KNN', 'LR', 'RF', 'XGB', 'LGBM'], index = range(4))

In [ ]:
#Support Vector Classifier

svc = SVC(random_state = 22, probability = True)
svc.fit(X_train_balanced, Y_train_balanced)
y_pred = svc.predict(X_test)
y_prob = svc.predict_proba(X_test)[:,1]

# Metrics
results.iloc[0, 0] = round(precision_score(y_test, y_pred), 2)
results.iloc[1, 0] = round(recall_score(y_test, y_pred), 2)
results.iloc[2, 0] = round(f1_score(y_test, y_pred), 2)
results.iloc[3, 0] = round(roc_auc_score(y_test, y_prob), 3)
svc_cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
print('')
print('-----------------------------------------------------')
print('')
print('Cross-validation scores with 5 folds:')
print('')
print(f"ROC AUC: {round(cross_val_score(svc, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'roc_auc').mean(), 3)}")
print(f"precision: {round(cross_val_score(svc, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'precision').mean(), 2)}")
print(f"recall: {round(cross_val_score(svc, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'recall').mean(), 2)}")
print(f"f1: {round(cross_val_score(svc, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'f1').mean(), 2)}")

# Visualize confusion matrix
plt.figure(figsize = (8, 5))
sns.heatmap(svc_cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15}, 
            yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# color palette for visualizations
colors = ['#f6f5f5','#d0e0f2','#1434A4','#08306b']
palette = sns.color_palette( palette = colors)

sns.palplot(palette, size =2.5)
plt.text(-0.75,-0.75,'Color Palette for this Visualization', {'font':'times new roman', 'size':25, 'weight':'bold'})
plt.text(-0.75,-0.64,'Most of the pages in this notebook will have the same colours.', {'font':'times new roman', 'size':18, 'weight':'normal'}, alpha = 0.8)
plt.show()

In [ ]:
#K-NEAREST NEIGHBORS Classifier

knn = KNeighborsClassifier()
knn.fit(X_train_balanced, Y_train_balanced)
y_pred = knn.predict(X_test)
y_prob = knn.predict_proba(X_test)[:,1]

# Metrics
results.iloc[0, 1] = round(precision_score(y_test, y_pred), 2)
results.iloc[1, 1] = round(recall_score(y_test, y_pred), 2)
results.iloc[2, 1] = round(f1_score(y_test, y_pred), 2)
results.iloc[3, 1] = round(roc_auc_score(y_test, y_prob), 3)
knn_cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
print('')
print('-----------------------------------------------------')
print('')
print('Cross-validation scores with 5 folds:')
print('')
print(f"ROC AUC: {round(cross_val_score(knn, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'roc_auc').mean(), 3)}")
print(f"precision: {round(cross_val_score(knn, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'precision').mean(), 2)}")
print(f"recall: {round(cross_val_score(knn, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'recall').mean(), 2)}")
print(f"f1: {round(cross_val_score(knn, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'f1').mean(), 2)}")

# Visualize confusion matrix
plt.figure(figsize = (8, 5))
sns.heatmap(knn_cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15}, 
            yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


In [ ]:
#Logistic Regression

lg = LogisticRegression(random_state = 22)
lg.fit(X_train_balanced, Y_train_balanced)
y_pred = lg.predict(X_test)
y_prob = lg.predict_proba(X_test)[:,1]

# Metrics
results.iloc[0, 2] = round(precision_score(y_test, y_pred), 2)
results.iloc[1, 2] = round(recall_score(y_test, y_pred), 2)
results.iloc[2, 2] = round(f1_score(y_test, y_pred), 2)
results.iloc[3, 2] = round(roc_auc_score(y_test, y_prob), 3)
lg_cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
print('')
print('-----------------------------------------------------')
print('')
print('Cross-validation scores with 5 folds:')
print('')
print(f"ROC AUC: {round(cross_val_score(lg, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'roc_auc').mean(), 3)}")
print(f"precision: {round(cross_val_score(lg, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'precision').mean(), 2)}")
print(f"recall: {round(cross_val_score(lg, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'recall').mean(), 2)}")
print(f"f1: {round(cross_val_score(lg, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'f1').mean(), 2)}")

# Visualize confusion matrix
plt.figure(figsize = (8, 5))
sns.heatmap(lg_cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15}, 
            yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Feature importance
f_imp = pd.DataFrame(columns = ['feature', 'importance (abs coef)'], index = range(13))
for i in range(len(f_imp.index)):
    f_imp.iloc[i, 0] = X_train_balanced.columns.to_list()[i]
f_imp['importance (abs coef)'] = abs(lg.coef_)[0]
f_imp = f_imp.sort_values('importance (abs coef)', ascending = False)
f_imp[0:12].style.background_gradient(cmap = 'Blues')

In [ ]:
#Random Forest Classifier

rf = RandomForestClassifier(random_state = 22, max_depth = 5)
rf.fit(X_train_balanced, Y_train_balanced)
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:,1]

# Metrics
results.iloc[0, 3] = round(precision_score(y_test, y_pred), 2)
results.iloc[1, 3] = round(recall_score(y_test, y_pred), 2)
results.iloc[2, 3] = round(f1_score(y_test, y_pred), 2)
results.iloc[3, 3] = round(roc_auc_score(y_test, y_prob), 3)
rf_cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
print('')
print('-----------------------------------------------------')
print('')
print('Cross-validation scores with 5 folds:')
print('')
print(f"ROC AUC: {round(cross_val_score(rf, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'roc_auc').mean(), 3)}")
print(f"precision: {round(cross_val_score(rf, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'precision').mean(), 2)}")
print(f"recall: {round(cross_val_score(rf, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'recall').mean(), 2)}")
print(f"f1: {round(cross_val_score(rf, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'f1').mean(), 2)}")

# Visualize confusion matrix
plt.figure(figsize = (8, 5))
sns.heatmap(rf_cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15},
           yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Feature importance
f_imp2 = pd.DataFrame(columns = ['feature', 'importance'], index = range(13))
for i in range(len(f_imp2.index)):
    f_imp2.iloc[i, 0] = X_train_balanced.columns.to_list()[i]
f_imp2['importance'] = rf.feature_importances_
f_imp2 = f_imp2.sort_values('importance', ascending = False)
f_imp2[0:12].style.background_gradient(cmap = 'Blues')

In [ ]:
#XGB Classifier

xgb = XGBClassifier(random_state = 22, max_depth = 5, objective = 'binary:logistic', eval_metric = 'logloss')
xgb.fit(X_train_balanced, Y_train_balanced)
y_pred = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)[:,1]

# Metrics
results.iloc[0, 4] = round(precision_score(y_test, y_pred), 2)
results.iloc[1, 4] = round(recall_score(y_test, y_pred), 2)
results.iloc[2, 4] = round(f1_score(y_test, y_pred), 2)
results.iloc[3, 4] = round(roc_auc_score(y_test, y_prob), 3)
xgb_cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
print('')
print('-----------------------------------------------------')
print('')
print('Cross-validation scores with 5 folds:')
print('')
print(f"ROC AUC: {round(cross_val_score(xgb, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'roc_auc').mean(), 3)}")
print(f"precision: {round(cross_val_score(xgb, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'precision').mean(), 2)}")
print(f"recall: {round(cross_val_score(xgb, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'recall').mean(), 2)}")
print(f"f1: {round(cross_val_score(xgb, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'f1').mean(), 2)}")

# Visualize confusion matrix
plt.figure(figsize = (8, 5))
sns.heatmap(xgb_cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15},
           yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Feature importance
f_imp3 = pd.DataFrame(columns = ['feature', 'importance'], index = range(13))
for i in range(len(f_imp3.index)):
    f_imp3.iloc[i, 0] = X_train_balanced.columns.to_list()[i]
f_imp3['importance'] = xgb.feature_importances_
f_imp3 = f_imp3.sort_values('importance', ascending = False)
f_imp3[0:12].style.background_gradient(cmap = 'Blues')

In [ ]:
#LGBM Classifier

lgbm = LGBMClassifier(random_state = 22, max_depth = 5, num_leaves = 50)
lgbm.fit(X_train_balanced, Y_train_balanced)
y_pred = lgbm.predict(X_test)
y_prob = lgbm.predict_proba(X_test)[:,1]

# Metrics
results.iloc[0, 5] = round(precision_score(y_test, y_pred), 2)
results.iloc[1, 5] = round(recall_score(y_test, y_pred), 2)
results.iloc[2, 5] = round(f1_score(y_test, y_pred), 2)
results.iloc[3, 5] = round(roc_auc_score(y_test, y_prob), 3)
lgbm_cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
print('')
print('-----------------------------------------------------')
print('')
print('Cross-validation scores with 5 folds:')
print('')
print(f"ROC AUC: {round(cross_val_score(lgbm, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'roc_auc').mean(), 3)}")
print(f"precision: {round(cross_val_score(lgbm, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'precision').mean(), 2)}")
print(f"recall: {round(cross_val_score(lgbm, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'recall').mean(), 2)}")
print(f"f1: {round(cross_val_score(lgbm, X_train_balanced, Y_train_balanced, cv = 5, scoring = 'f1').mean(), 2)}")

# Visualize confusion matrix
plt.figure(figsize = (8, 5))
sns.heatmap(lgbm_cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15},
           yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Feature importance
f_imp4 = pd.DataFrame(columns = ['feature', 'importance'], index = range(13))
for i in range(len(f_imp4.index)):
    f_imp4.iloc[i, 0] = X_train_balanced.columns.to_list()[i]
f_imp4['importance'] = lgbm.feature_importances_
f_imp4 = f_imp4.sort_values('importance', ascending = False)
f_imp4[0:12].style.background_gradient(cmap = 'Blues')

In [ ]:
plt.figure(figsize = (10, 7))
sns.heatmap(results[results.columns.to_list()].astype(float), cmap = 'Blues', annot = True, linewidths = 1, cbar = False, annot_kws = {'fontsize': 12},
           yticklabels = ['Precision', 'Recall', 'F1', 'ROC AUC'])
sns.set(font_scale = 1.5)
plt.yticks(rotation = 0)
plt.show()

In [ ]:
### Working directories and exporting files.
# Check working directory.
os.getcwd()
# Set wd to correct location - the main submission folder.
root = '/Users/apple/Documents/TU/second semester/Machine learning/STROKE'
# Set working directory.
os.chdir(root)
# Export visuals wd.
os.chdir(root + '//ICA  MACHINE LEARNING//ELEMENT 1//SCRIPTS')



#%% Export files.
# # Import libraries.
import os
# from joblib import dump, load
# import matplotlib.pyplot as plt
# # Check working directory.
# os.getcwd()
# # Set wd to correct location - the main submission folder.
# root = 'C://A0222219_Lauren_McGinney'
31
# # Set working directory.
# os.chdir(root)
# ### Export dataframes.
# # Set wd.
os.chdir(root + '//ICA  MACHINE LEARNING//ELEMENT 1//SCRIPTS')
#data.to_csv('data.csv', index=False)
#data1.to_csv('data1.csv', index=False)
#data2.to_csv('data2.csv', index=False)
# # Dataframes cn be loaded using the usual read_csv function. For example:
# # data1 = pd.read_csv("data1.csv")
# ### Export ML models.
# # Set wd.
# os.chdir(root + '//outputs//models')
# Example of saving model.
# dump(model1, 'model1.joblib')
# # Models can be loaded by using the load function. For example:
# # model1 = load('model1.joblib')
# https://scikit-learn.org/stable/modules/model_persistence.html#model-persistence
# Reset wd to root.
os.chdir(root)